In [ ]:
# pip install openai

In [ ]:
# pip install python-dotenv

In [ ]:
# pip install panel

# Setup

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('keys.env')) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# OrderBot

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.2, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

def collect_messages(_):
    prompt = inp.value_input
    # append user prompt
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    # append response from system
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    # store as context
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    pannels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
import panel as pn # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You will follow the steps accordingly whenever you meet a customer: \
1. Find the current time.
2. You must greet the customer according to the time and introduce yourself in a kind manner. \
3. Ask the customer for their name and always refer to them by their name. \
4. take their order. \
5. Then ask if the order is for a pickup or delivery. \
6. You wait to collect the entire order, then summarize it
7. check for a final time if the customer wants to add anything else. \
8. If it's a delivery, you ask for an address and when the delivery date should be. \
9. Ask the customer if the payment method is by cash or card.\
10. If it's card, collect payment. \
Always check that all the conditions have been satisfied before you proceed to the next step. \
If the customer wants to cancel their order, give them a warm good bye. \
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational and friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)